In [ ]:
"""
This code is for math research with friends.

Let G be a spider graph with k legs of length m.
We play the chip firing game with k * m meny chips.
The chips are named [1,2,...,k*m].

What is a configuration?
Its how im representing the state of G while under the chip firing process.
Its a (2 by m * k) - matrix.
Every row corresponds to a chip.
The i-th row records all information about the i-th chip.
in 1st colum: branch name. we record the name of the leg/branch that chip-i calls home
in 2nd colum:  level name. we have the level on the leg, so distance from center

very import! name =/= index
name is human numbering 1,2,3...
index is computer numbering 0,1,2,...
"""

In [1]:
import time  # to measure speed of program

import numpy as np
from scipy.special import binom
from itertools import combinations

In [ ]:
start_time = time.perf_counter()

k = 3  # number of branches
m = 3  # number of chips per branch, total chips = k*m

# here we pre-allocate the space needed to record all Rank 1 configurations
Rank1 = int(binom(m * k, k))
listOfConfigurations = np.zeros(shape=(Rank1, 2, m * k), dtype=np.uint8)
L = 0  # length of listOfConfigurations, rather index of next to be added
chips = np.arange(m * k)  # list of chip indexes, not the names
for combs_i in combinations(chips, k):
    # the first chip of combs_i should be the smallest and go to branch 1
    # the last chip of combs_i should be the biggest and go to branch k
    listOfConfigurations[L, 0, combs_i] = np.arange(1, k + 1, dtype=np.uint8)
    listOfConfigurations[L, 1, combs_i] = 1
    L += 1

# start loop with Rank 2 until finished
maxRank = int(m * (m + 1) / 2 + k * (m - 1) * m * (m + 1) / 6)  # height of decision tree
print(f"{maxRank=}")

# pre_alo = 10**7  # k=3, m=4
# pre_alo = 15000  # for k=2, m=4
# pre_alo = 500000  # for k=2, m=5
# pre_alo = 10**10  # just a guess, for k=2, m=6
# pre_alo = 10**12  # bruh, this is the max my computer can handle, for k=2, m=8
pre_alo = 10**4
tempListOfConfigurations = np.zeros((pre_alo, 2, m * k), dtype=np.uint8)
L = 0

for Rank in range(2, maxRank + 1):
    print(f'{Rank=}')  # %%%%%%%%%%%%%%
    # for each configuration
    # figure out which vertices can be fired and how many ways
    # keep track of new configurations
    for configuration in listOfConfigurations:
        columns, inverse, counts = np.unique(configuration, axis=1, return_inverse=True, return_counts=True)

        # Angel said:
        # the way np.unique works is by using a sorting algorithm
        # the sorting algorithm is unstable so [0,0] might not always be first.
        # unless you add sorted=True, but that'll make the code slower
        # Annika says:
        # for some reason my computer didn't like the sorted thing anyways so I tossed it
        for i in np.where(counts >= 2)[0]:
            chip_indices = np.where(inverse == i)[0]
            if i == 0 and columns[0, 0] == 0:
                # fire the center node
                ''' the k=2 case '''
                # for i_1, i_2 in combinations(chip_indices, 2):
                #     tempListOfConfigurations[L] = configuration
                #     tempListOfConfigurations[L, 0, i_1] = 1  # name of branch
                #     tempListOfConfigurations[L, 1, i_1] = 1  # name of level
                #     tempListOfConfigurations[L, 0, i_2] = 2  # name of branch
                #     tempListOfConfigurations[L, 1, i_2] = 1  # name of level
                #     L += 1
                ''' the general case '''
                for ids in combinations(chip_indices, k):
                    # ids is a list (size k) of indexes of chips
                    # ids should be sorted already
                    # add the fired chips to the current configuration
                    tempListOfConfigurations[L] = configuration
                    for [i, chip_id] in enumerate(ids):
                        # the first chip of combs_i should be the smallest and go to branch 1
                        # the last chip of combs_i should be the biggest and go to branch k
                        # recall name of branch =/= index of branch
                        tempListOfConfigurations[L, 0, chip_id] = i + 1  # name of branch
                        tempListOfConfigurations[L, 1, chip_id] = 1
                    L += 1
            else:
                # fire a non-center node
                for i_1, i_2 in combinations(chip_indices, 2):
                    tempListOfConfigurations[L] = configuration
                    tempListOfConfigurations[L, 1, i_2] += 1  # level goes up for bigger one
                    tempListOfConfigurations[L, 1, i_1] -= 1  # level goes down for smaller one
                    # if smaller one entered level 0, then set branch to 0
                    if tempListOfConfigurations[L, 1, i_1] <= 0:
                        tempListOfConfigurations[L, 0, i_1] = 0
                    L += 1  # length of newListOfConfigurations
    # print('deleting duplicates')  # %%%%%%%%%%%%%%
    listOfConfigurations = np.unique(tempListOfConfigurations[:L], axis=0)
    # print(f'Out of {L=:1.3e} configs, {listOfConfigurations.shape[0]:1.3e} are unique')
    L = 0



print(f'number of reachable configurations: {listOfConfigurations.shape[0]}')
# print(listOfConfigurations.shape[0])

# Convert configurations to tableau
# Record number of non-standard young tableau
# and Print the resulting non-standard young tableau
N_syt = 0
for configuration in listOfConfigurations:
    # convert configuration into a tableau
    tableau = np.zeros((k, m))
    for c in range(m * k):  # this is index, recall index of chip =/= name of chip
        # name of branch =/= index of branch
        # name of level =/= index of level
        branch = int(configuration[0, c]) - 1  # index
        level = int(configuration[1, c]) - 1  # index
        tableau[branch, level] = c + 1  # name of chip
    # print(tableau, '\n')
    # check if tableau is standard young tableau
    sorted_cols = np.all(tableau[:-1, :] <= tableau[1:, :])
    # sorted_rows = np.all(tableau[:, :-1] <= tableau[:, 1:])
    is_syt = sorted_cols  # and sorted_rows
    if not is_syt:
        N_syt += 1
        print('\n %%%%%%%%%%%%%% \n', tableau, '\n %%%%%%%%%%%%%% \n')
    # else:
        print(tableau, '\n')

print(f'number of none standard young tableau: {N_syt}')

end_time = time.perf_counter()
elapsed_time = end_time - start_time
print(f"Execution time: {elapsed_time:.6f} seconds")

In [6]:
def runGame(k,m):
    start_time = time.perf_counter()

    print(f"(k,m) = ({k},{m})")
    
    # k = number of branches
    # m = number of chips per branch, total chips = k*m
    
    # here we pre-allocate the space needed to record all Rank 1 configurations
    Rank1 = int(binom(m * k, k))
    listOfConfigurations = np.zeros(shape=(Rank1, 2, m * k), dtype=np.uint8)
    L = 0  # length of listOfConfigurations, rather index of next to be added
    chips = np.arange(m * k)  # list of chip indexes, not the names
    for combs_i in combinations(chips, k):
        # the first chip of combs_i should be the smallest and go to branch 1
        # the last chip of combs_i should be the biggest and go to branch k
        listOfConfigurations[L, 0, combs_i] = np.arange(1, k + 1, dtype=np.uint8)
        listOfConfigurations[L, 1, combs_i] = 1
        L += 1
    
    # start loop with Rank 2 until finished
    maxRank = int(m * (m + 1) / 2 + k * (m - 1) * m * (m + 1) / 6)  # height of decision tree
    print(f"{maxRank=}")

    if k == 1:
        pre_alo = 1000
    elif k == 2 and m<=4:
        pre_alo = 15000  # for k=2, m=4
    elif (k,m) == (2,5):
        pre_alo = 500000  # for k=2, m=5
    elif (k,m) == (2,6):
        pre_alo = 10**10  # just a guess, for k=2, m=6
    elif (k,m) == (2,7) or (k,m) == (2,8):
        pre_alo = 10**12  # for k=2, m=8
    elif k == 3 and m<=3:
        pre_alo = 10**4
    elif (k,m) == (3,4):
        pre_alo = 10**7  # k=3, m=4
    else:
        print("too big")
        
    tempListOfConfigurations = np.zeros((pre_alo, 2, m * k), dtype=np.uint8)
    L = 0
    
    for Rank in range(2, maxRank + 1):
        # print(f'{Rank=}')  # %%%%%%%%%%%%%%
        # for each configuration
        # figure out which vertices can be fired and how many ways
        # keep track of new configurations
        for configuration in listOfConfigurations:
            columns, inverse, counts = np.unique(configuration, axis=1, return_inverse=True, return_counts=True)
    
            # Angel said:
            # the way np.unique works is by using a sorting algorithm
            # the sorting algorithm is unstable so [0,0] might not always be first.
            # unless you add sorted=True, but that'll make the code slower
            # Annika says:
            # for some reason my computer didn't like the sorted thing anyways so I tossed it
            for i in np.where(counts >= 2)[0]:
                chip_indices = np.where(inverse == i)[0]
                if i == 0 and columns[0, 0] == 0:
                    # fire the center node
                    ''' the k=2 case '''
                    if k == 2:
                        for i_1, i_2 in combinations(chip_indices, 2):
                            tempListOfConfigurations[L] = configuration
                            tempListOfConfigurations[L, 0, i_1] = 1  # name of branch
                            tempListOfConfigurations[L, 1, i_1] = 1  # name of level
                            tempListOfConfigurations[L, 0, i_2] = 2  # name of branch
                            tempListOfConfigurations[L, 1, i_2] = 1  # name of level
                            L += 1
                    ''' the general case '''
                    if k!=2:
                        for ids in combinations(chip_indices, k):
                            # ids is a list (size k) of indexes of chips
                            # ids should be sorted already
                            # add the fired chips to the current configuration
                            tempListOfConfigurations[L] = configuration
                            for [i, chip_id] in enumerate(ids):
                                # the first chip of combs_i should be the smallest and go to branch 1
                                # the last chip of combs_i should be the biggest and go to branch k
                                # recall name of branch =/= index of branch
                                tempListOfConfigurations[L, 0, chip_id] = i + 1  # name of branch
                                tempListOfConfigurations[L, 1, chip_id] = 1
                            L += 1
                else:
                    # fire a non-center node
                    for i_1, i_2 in combinations(chip_indices, 2):
                        tempListOfConfigurations[L] = configuration
                        tempListOfConfigurations[L, 1, i_2] += 1  # level goes up for bigger one
                        tempListOfConfigurations[L, 1, i_1] -= 1  # level goes down for smaller one
                        # if smaller one entered level 0, then set branch to 0
                        if tempListOfConfigurations[L, 1, i_1] <= 0:
                            tempListOfConfigurations[L, 0, i_1] = 0
                        L += 1  # length of newListOfConfigurations
        # print('deleting duplicates')  # %%%%%%%%%%%%%%
        listOfConfigurations = np.unique(tempListOfConfigurations[:L], axis=0)
        # print(f'Out of {L=:1.3e} configs, {listOfConfigurations.shape[0]:1.3e} are unique')
        L = 0
    
    
    
    print(f'number of reachable configurations: {listOfConfigurations.shape[0]}')
    # print(listOfConfigurations.shape[0])
    
    # Convert configurations to tableau
    # Record number of non-standard young tableau
    # and Print the resulting non-standard young tableau
    N_syt = 0
    for configuration in listOfConfigurations:
        # convert configuration into a tableau
        tableau = np.zeros((k, m))
        for c in range(m * k):  # this is index, recall index of chip =/= name of chip
            # name of branch =/= index of branch
            # name of level =/= index of level
            branch = int(configuration[0, c]) - 1  # index
            level = int(configuration[1, c]) - 1  # index
            tableau[branch, level] = c + 1  # name of chip
        # print(tableau, '\n')
        # check if tableau is standard young tableau
        sorted_cols = np.all(tableau[:-1, :] <= tableau[1:, :])
        # sorted_rows = np.all(tableau[:, :-1] <= tableau[:, 1:])
        is_syt = sorted_cols  # and sorted_rows
        if not is_syt:
            N_syt += 1
            print('\n', tableau, '\n')
        # else:
            # print(tableau, '\n')
    
    print(f'number of non-standard young tableau: {N_syt}')
    
    end_time = time.perf_counter()
    elapsed_time = end_time - start_time
    print(f"Execution time: {elapsed_time:.6f} seconds")

In [7]:
for k in range(1,3):
    for m in range(1,6):
        print('\n')
        print('\n')
        print('\n')
        runGame(k,m)







(k,m) = (1,1)
maxRank=1
number of reachable configurations: 1
number of non-standard young tableau: 0
Execution time: 0.000396 seconds






(k,m) = (1,2)
maxRank=4
number of reachable configurations: 0
number of non-standard young tableau: 0
Execution time: 0.005689 seconds






(k,m) = (1,3)
maxRank=10
number of reachable configurations: 0
number of non-standard young tableau: 0
Execution time: 0.003797 seconds






(k,m) = (1,4)
maxRank=20
number of reachable configurations: 0
number of non-standard young tableau: 0
Execution time: 0.011071 seconds






(k,m) = (1,5)
maxRank=35
number of reachable configurations: 0
number of non-standard young tableau: 0
Execution time: 0.072881 seconds






(k,m) = (2,1)
maxRank=1
number of reachable configurations: 1
number of non-standard young tableau: 0
Execution time: 0.000354 seconds






(k,m) = (2,2)
maxRank=5
number of reachable configurations: 2
number of non-standard young tableau: 0
Execution time: 0.003356 seconds






(k,m

In [8]:
for m in range(1,6):
    print('\n')
    print('\n')
    print('\n')
    runGame(3,m)







(k,m) = (3,1)
maxRank=1
number of reachable configurations: 1
number of non-standard young tableau: 0
Execution time: 0.002201 seconds






(k,m) = (3,2)
maxRank=6
number of reachable configurations: 5
number of non-standard young tableau: 0
Execution time: 0.028732 seconds






(k,m) = (3,3)
maxRank=18
number of reachable configurations: 47

 [[1. 4. 5.]
 [2. 3. 7.]
 [6. 8. 9.]] 


 [[1. 4. 5.]
 [2. 3. 8.]
 [6. 7. 9.]] 


 [[1. 4. 6.]
 [2. 3. 7.]
 [5. 8. 9.]] 


 [[1. 4. 6.]
 [2. 3. 8.]
 [5. 7. 9.]] 


 [[1. 4. 7.]
 [2. 3. 8.]
 [5. 6. 9.]] 

number of non-standard young tableau: 5
Execution time: 2.615305 seconds






(k,m) = (3,4)
maxRank=40
number of reachable configurations: 762

 [[ 1.  2.  3.  4.]
 [ 5.  8.  9. 10.]
 [ 6.  7. 11. 12.]] 


 [[ 1.  2.  3.  4.]
 [ 5.  8.  9. 11.]
 [ 6.  7. 10. 12.]] 


 [[ 1.  2.  3.  5.]
 [ 4.  8.  9. 10.]
 [ 6.  7. 11. 12.]] 


 [[ 1.  2.  3.  5.]
 [ 4.  8.  9. 11.]
 [ 6.  7. 10. 12.]] 


 [[ 1.  2.  3.  6.]
 [ 4.  8.  9. 10.]
 [ 5.  7. 1

MemoryError: Unable to allocate 279. GiB for an array with shape (10000000000, 2, 15) and data type uint8